In [1]:
%load_ext autoreload
%autoreload 2

# Import library

In [51]:
import numpy as np
import math
# from . import calc_util as cu
import sys
sys.path.append('src')
import enex_analysis as enex
import dartwork_mpl as dm
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.ticker as ticker
dm.use_dmpl_style()


# 0. 준비

In [3]:
## Fontsize 지정
plt.rcParams['font.size'] = 9

fs = {
    'label': dm.fs(0),
    'tick': dm.fs(-1.5),
    'legend': dm.fs(-2.0),
    'subtitle': dm.fs(-0.5),
    'cbar_tick': dm.fs(-2.0),
    'cbar_label': dm.fs(-2.0),
    'cbar_title': dm.fs(-1),
    'setpoint': dm.fs(-1),
    'text': dm.fs(-3.0),
            }

pad = {
    'label': 6,
    'tick': 5,
}

LW = np.arange(0.25, 3.0, 0.25)

# 1. 단일 운전점 최적화 & 사이클 도식

In [4]:
# =============================================================================
# 1) 단일 운전점 최적화 & 사이클 도식
# =============================================================================
GSHPB = enex.GroundSourceHeatPumpBoiler2(
    refrigerant  = 'R410A',
    disp_cmp     = 0.00005, #0.00001 ~ 0.00011m³/rev
    eta_cmp_isen = 0.7,
    eta_cmp_dV   = 0.85,
    T_f_bh_in    = 15.0,
    Tg           = 15.0,
    UA_HX_tank       = 500,  # W/K
    UA_HX_water_loop = 500,  # W/K
    D_b = 0,                 # Borehole depth [m]
    H_b = 200,               # Borehole height [m]
    r_b = 0.08,              # Borehole radius [m]
    R_b = 0.108,             # Effective borehole thermal resistance [mK/W]
    dV_f = 24,               # Volumetric flow rate of fluid [L/min]
    k_g   = 2.0,
    c_g   = 800,
    rho_g = 2000,
    E_pmp = 200,
)

ref_loop_result = GSHPB._find_ref_loop_optimal_operation(T_w_tank=60, Q_load=-6000)

print(
    f'T1: {enex.K2C(ref_loop_result["T1"]):.2f} °C,\n'
    f'T2: {enex.K2C(ref_loop_result["T2"]):.2f} °C,\n'
    f'T3: {enex.K2C(ref_loop_result["T3"]):.2f} °C,\n'
    f'T4: {enex.K2C(ref_loop_result["T4"]):.2f} °C,\n'
    f'E_cmp: {ref_loop_result["E_cmp"]:.1f} W,\n'
    f'cmp_rps: {ref_loop_result["cmp_rps"]:.2f} rps,\n'
    f'cmp_rpm: {ref_loop_result["cmp_rps"]*60:.0f} rpm,\n'
)

GSHPB.plot_cycle_diagrams(result=ref_loop_result)

T1: 8.09 °C,
T2: 95.26 °C,
T3: 62.25 °C,
T4: 8.03 °C,
E_cmp: 1956.6 W,
cmp_rps: 22.06 rps,
cmp_rpm: 1324 rpm,



/home/habin/Codes/En_system_Ex_analysis/src/enex_analysis/HeatPump_model.py:1117: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  axi.set_ylim(*ylims[idx])


## 1.1 Constant

In [5]:
c_w = 4186  # J/kgK
rho_w = 1000  # kg/m3

# 2. 장기 시뮬레이션 설정

In [6]:
# =============================================================================
# 2) 장기 시뮬레이션 설정
# =============================================================================
GSHPB = enex.GroundSourceHeatPumpBoiler2(
    refrigerant  = 'R410A',
    disp_cmp     = 0.00005,
    eta_cmp_isen = 0.7,
    eta_cmp_dV   = 0.85,
    T_f_bh_in    = 15.0,
    Tg           = 15.0,
    UA_HX_tank       = 500,  # W/K
    UA_HX_water_loop = 500,  # W/K
    D_b = 0,                 # Borehole depth [m]
    H_b = 200,               # Borehole height [m]
    r_b = 0.08,              # Borehole radius [m]
    R_b = 0.108,             # Effective borehole thermal resistance [mK/W]
    dV_f = 24,               # Volumetric flow rate of fluid [L/min]
    k_g   = 2.0,
    c_g   = 800,
    rho_g = 2000,
    E_pmp = 200,
)

# 시뮬레이션 타임축
simulation_period_sec = 10 * enex.d2h * enex.h2s  # 10일 [s]
dt = 1 * enex.m2s  # 1분 간격
start_time = 0
end_time = simulation_period_sec
time = np.arange(start_time, end_time, dt)
tN = len(time)
print(tN)

# Load profile 설정 (W) #####################################
''' 
Q_load_day1: 하루 24시간 급탕 부하 시간대 6~7 h, 18~19 h
'''

Q_serv_schedule_day1 = [0, 0, 0, 0, 0, 0, # 0 h ~ 6 h
                        1, 0, 0, 0, 0, 0, # 7 h ~ 12 h
                        0, 0, 0, 0, 0, 1, # 13 h ~ 18 h
                        0, 0, 0, 0, 0, 0,]  # 19 h ~ 24 h

Q_serv_schedule_day1 = np.repeat(Q_serv_schedule_day1, int(enex.h2s/dt)) # time step이 1분이므로, 1시간당 60 스텝으로 변경

# 하중 및 탱크 목표온도

# 결과 버퍼


T_w_serv  = 45
T_w_sup   = 15
dV_w_serv = 8 * enex.L2m3/ enex.m2s
T0        = 0                     

# 초기 상태/상수
Q_bh_unit_purse = np.zeros(tN)
Q_bh_unit_old = 0

T0_K = enex.C2K(T0)                                    # 기준 온도 [K]
P0   = 101325                                          # 기준 압력 [kPa]
h0   = CP.PropsSI('H', 'P', P0, 'T', T0_K, GSHPB.ref)  # 기준 엔탈피 [J/kg]
s0   = CP.PropsSI('S', 'P', P0, 'T', T0_K, GSHPB.ref)  # 기준 엔트로피 [J/kgK]


# =============================================================================
# 2-1) 제어 로직 및 탱크 동특성 파라미터 설정
# =============================================================================
# ... (기존 GSHPB 객체 생성, time 배열 생성 등) ...

# --- (A) 탱크 동특성 파라미터 ---
r0 = 0.2; H = 0.8
UA_tank = enex.calc_simple_tank_UA(r0=r0, H=H) # W/K
V_tank = math.pi * r0**2 * H
C_tank = enex.c_w * enex.rho_w * V_tank # J/K, 탱크의 열용량

# --- (B) 온도 기반 제어(Thermostat) 파라미터 ---
T_w_tank_setpoint = 60.0  # 목표 상한 온도 (°C)
T_w_tank_lower_bound = 50.0 # 하한 온도 (°C)

# --- (C) 탱크 온도 상태배열 생성 (°C) ---
T_w_tank_init = 55.0  # 초기 탱크 온도 (상한과 하한 사이에서 시작)
T_w_tank = np.empty(tN)
T_w_tank[0] = T_w_tank_init

# --- (D) 급탕 사용 스케줄 생성 ---
serv_sched_day = np.array(Q_serv_schedule_day1, dtype=float)
serv_sched_tot = serv_sched_day[np.mod(np.arange(tN), int(24*enex.h2s/dt))] # serv)sched: 인덱스

# --- (C) 탱크 온도 상태배열 생성 (°C) ---
T_w_tank_init = 55.0  # 기존에 scalar로 있던 초기 탱크온도(예: 60)
T_w_tank = np.empty(tN)
T_w_tank[0] = T_w_tank_init

# --- (D) (선택) 부하를 루프에서 일관되게 계산할 것이므로, 기존 Q_load는 참조만 하거나 건너뜀 ---
# Q_load는 컨트롤/비교용으로 남겨도 되지만, 루프 내부에서 q_load_n을 다시 계산해 사용합니다.

14400


# 2. Function

In [167]:
# =============================================================================
# 4. 시뮬레이션 실행 함수 및 로직 (개선된 버전)
# =============================================================================
def run_simulation(gshpb, config, serv_sched):
    # --- 초기화 ---
    time = config['time']
    tN = len(time)
    results_df = pd.DataFrame(index=range(tN), columns=config['results_keys'])
    
    # 동적 상태 변수 초기화
    T_w_tank_series = np.full(tN, np.nan)
    T_w_tank_series[0] = config['T_w_tank_init']
    Q_bh_unit_pulse = np.zeros(tN)
    Q_bh_unit_old = 0
    is_on_prev_step = False

    # --- 시뮬레이션 루프 ---
    for n in tqdm(range(tN), desc="Simulating"):
        # ⭐️ 1. 매 스텝마다 결과를 담을 '결과 수집기' 딕셔너리 생성
        step_results = {}
        
        # 현재 값들을 수집기에 추가
        T_w_tank_n = T_w_tank_series[n]
        step_results['T_w_tank'] = T_w_tank_n

        # 2. 제어 상태 결정 (수집기 전달)
        is_on_n, q_load_n = update_control_state(
            T_w_tank_n, is_on_prev_step, serv_sched[n], config, step_results
        )
        
        # 3. 히트펌프 운전점 계산
        ref_loop_result = gshpb._find_ref_loop_optimal_operation(T_w_tank=T_w_tank_n, Q_load=q_load_n)
        if ref_loop_result is None:
            ref_loop_result = gshpb._off_result(T_w_tank_n)
            is_on_n = False
        
        # 계산 결과를 수집기에 추가
        step_results.update(ref_loop_result)
        step_results['is_on'] = is_on_n

        # 4. 지중 온도 업데이트 (수집기 전달)
        Q_bh_unit_pulse, Q_bh_unit_old = update_ground_temperature(
            n, ref_loop_result, Q_bh_unit_pulse, Q_bh_unit_old, gshpb, config, step_results
        )
        
        # 5. 다음 스텝 탱크 온도 계산
        if n < tN - 1:
            Q_tank_in = -ref_loop_result.get('Q_ref_tank', 0.0)
            Q_net = Q_tank_in - step_results['total_loss'] # 수집기에서 값 사용
            T_w_tank_series[n+1] = T_w_tank_n + (Q_net / config['C_tank']) * config['dt']

        gshpb.T_f_bh_in_K = step_results['T_f_bh_in'] # 수집기에서 값 사용  
        is_on_prev_step = is_on_n
        
        # ⭐️ 6. 매우 깔끔해진 결과 저장 단계
        store_results(results_df, n, step_results)
        
    return results_df


# ⭐️ 함수 시그니처와 내용 변경
def update_control_state(T_w_tank_n, is_on_prev, schedule_n, config, results_container):
    """결과를 results_container에 직접 저장하고, 계산에 필요한 값만 반환합니다."""
    # ... (손실 계산 로직은 동일) ...
    Q_env_loss = config['UA_tank'] * (T_w_tank_n - config['T0'])
    den = max(1e-6, enex.C2K(T_w_tank_n) - enex.C2K(config['T_w_sup']))
    alp = min(1.0, max(0.0, enex.C2K(config['T_w_serv']) - enex.C2K(config['T_w_sup'])) / den)
    dV_w_serv = schedule_n * config['dV_w_serv_m3s']
    dV_w_sup_tank = alp * dV_w_serv
    Q_use_loss = schedule_n * (c_w * rho_w * dV_w_sup_tank * (T_w_tank_n - config['T_w_sup']))
    total_loss = Q_env_loss + Q_use_loss
    
    # On/Off 결정
    if T_w_tank_n < config['T_w_tank_lower_bound']: is_on = True
    elif T_w_tank_n > config['T_w_tank_setpoint']: is_on = False
    else: is_on = is_on_prev
    q_load_n = -config['heater_capacity'] if is_on else 0.0
    
    # ⭐️ 계산된 결과를 딕셔너리에 직접 추가
    results_container['T_w_tank'] = T_w_tank_n
    results_container['is_on'] = is_on
    results_container['dV_w_serv'] = dV_w_serv
    results_container['Q_env_loss'] = Q_env_loss
    results_container['Q_use_loss'] = Q_use_loss
    results_container['total_loss'] = total_loss
    
    # ⭐️ 다음 '계산'에 필요한 값만 반환
    return is_on, q_load_n

# ⭐️ 함수 시그니처와 내용 변경
def update_ground_temperature(n, ref_res, Q_bh_pulse, Q_bh_unit_old, gshpb, config, results_container):
    """결과를 results_container에 직접 저장하고, 상태 업데이트에 필요한 값만 반환합니다."""
    # ... (지중 온도 계산 로직은 동일) ...
    Q_bh_unit = (ref_res.get('Q_ref_HX', 0.0) - gshpb.E_pmp) / gshpb.H_b if ref_res.get('is_on') else 0.0
    
    if abs(Q_bh_unit - Q_bh_unit_old) > 1e-6:
        Q_bh_pulse[n] = Q_bh_unit - Q_bh_unit_old
        Q_bh_unit_old_new = Q_bh_unit
    else:
        Q_bh_unit_old_new = Q_bh_unit_old

    # ... (g-function 계산 등 나머지 로직) ...
    Q_bh_unit = (ref_res.get('Q_ref_HX', 0.0) - gshpb.E_pmp) / gshpb.H_b if ref_res.get('is_on') else 0.0
    
    if abs(Q_bh_unit - Q_bh_unit_old) > 1e-6:
        Q_bh_pulse[n] = Q_bh_unit - Q_bh_unit_old
        Q_bh_unit_old = Q_bh_unit
    
    pulses_idx = np.flatnonzero(Q_bh_pulse[:n+1])
    dQ = Q_bh_pulse[pulses_idx]
    tau = config['time'][n] - config['time'][pulses_idx]
    
    # g-function 계산은 여전히 루프가 필요
    g_n = np.array([enex.G_FLS(t, gshpb.k_g, gshpb.alp_g, gshpb.r_b, gshpb.H_b) for t in tau])
    g_conv = np.sum(g_n)
    
    dT_bh = np.dot(dQ, g_n)
    T_bh = gshpb.Tg - dT_bh
    
    T_f_bh = T_bh - Q_bh_unit * gshpb.R_b
    Q_g_total = Q_bh_unit * gshpb.H_b
    T_f_bh_in = T_f_bh - Q_g_total / (2 * c_w * rho_w * gshpb.dV_f)
    T_f_bh_out = T_f_bh + Q_g_total / (2 * c_w * rho_w * gshpb.dV_f)
    
    T0_K = enex.C2K(config['T0'])
    Xg = (1 - T0_K / gshpb.Tg) * Q_g_total
    Xb = (1 - T0_K / T_bh) * Q_g_total
    Xc_g = Xg - Xb
    
    # ⭐️ 계산된 결과를 딕셔너리에 직접 추가
    results_container['T_bh'] = T_bh
    results_container['T_f_bh_in'] = T_f_bh_in
    results_container['T_f_bh_out'] = T_f_bh_out
    results_container['Q_bh_unit'] = Q_bh_unit
    results_container['g_conv'] = g_conv
    results_container['Xc_g'] = Xc_g
    results_container['dQ'] = Q_bh_pulse # dQ는 배열이지만, 필요하다면 n번째 값을 저장
    
    T_w_tank_n = results_container['T_w_tank']
    
    E_cmp = results_container.get('E_cmp', 0.0)
    Q_ref_tank = results_container.get('Q_ref_tank', 0.0)
    T3_K = results_container.get('T3', enex.C2K(T_w_tank_n))
    
    results_container['X_cmp'] = E_cmp
    results_container['X_pmp'] = gshpb.E_pmp if results_container.get('is_on') else 0.0
    results_container['X_ref_tank'] = Q_ref_tank * (1 - T0_K / T3_K)
    results_container['COP'] = abs(Q_ref_tank) / (E_cmp + gshpb.E_pmp) if (E_cmp + gshpb.E_pmp) > 0 else 0.0
    # ⭐️ 다음 루프에서 상태 유지를 위해 필요한 값만 반환
    return Q_bh_pulse, Q_bh_unit_old_new


# ⭐️ 완전히 새로워지고 단순해진 함수
def store_results(df, n, results_container, ):
    """
    '결과 수집기' 딕셔너리를 받아 DataFrame의 n번째 행에 저장합니다.
    엑서지처럼 여기서 추가 계산이 필요한 항목들도 처리합니다.
    """
    # 2. 수집기의 모든 값을 DataFrame에 한 번에 저장
    for key, value in results_container.items():
        # df의 columns에 해당 key가 있을 경우에만 저장 (안전장치)
        if key in df.columns:
            df.loc[n, key] = value

# 3. 시뮬레이션 설정

In [168]:
# =============================================================================
# 5. 메인 실행 블록
# =============================================================================
# --- 1. 시뮬레이션 설정 (중앙 관리) ---
config = {
        'simulation_period_sec': 2 * enex.d2h * enex.h2s,
        'heater_capacity': 7000.0, # W
        'dt': 1 * enex.m2s, # 1분
        'T_w_tank_setpoint': 60.0,
        'T_w_tank_lower_bound': 50.0,
        'T_w_tank_init': 55.0,
        'T0': 0.0,
        'T_w_serv': 45.0,
        'T_w_sup': 15.0,
        'dV_w_serv_m3s': 5 * enex.L2m3 / enex.m2s,
        'results_keys': [
            'T_bh', 'T_f_bh_in', 'T_f_bh_out', 'T_w_tank',
            'COP', 'g_conv',
            'Q_bh_unit',
            'Q_ref_tank', 'Q_ref_HX',
            'Q_env_loss', 'Q_use_loss', 'total_loss',
            'Q_ref',
            'is_on',
            'E_cmp', 'E_pmp',
            'Xin_g', 'Xout_g', 'Xc_g',
            'X_ref_tank'
            'Xin_GHE', 'Xout_GHE', 'Xc_GHE',
            'Xin_HX', 'Xout_HX', 'Xc_HX',
            'Xin_r', 'Xout_r', 'Xc_r',
            'Xin_tank', 'Xout_tank', 'Xc_tank',
            'Xc_mix',
            'X_eff',
            'dV_w_serv',
            # 필요한 다른 결과 키들 추가...
        ]
    }
# --- 2. 시간 및 부하 프로파일 생성 ---
config['time'] = np.arange(0, config['simulation_period_sec'], config['dt'])
tN = len(config['time'])

serv_sched_hourly = [0]*6 + [1]*1 + [0]*11 + [1]*1 + [0]*5 # 6-7h, 18-19h
serv_sched_minutely = np.repeat(serv_sched_hourly, int(enex.h2s / config['dt']))
num_days = int(np.ceil(tN / len(serv_sched_minutely)))
serv_sched = np.tile(serv_sched_minutely, num_days)[:tN]

# --- 3. 모델 및 기타 파라미터 초기화 ---
gshpb_model = enex.GroundSourceHeatPumpBoiler2(
    disp_cmp=0.00005, eta_cmp_isen=0.7, T_f_bh_in=15.0, Tg=15.0
)

tank_params = {'r0': 0.2, 'H': 0.8}
config['UA_tank'] = 3.5 # enex.calc_simple_tank_UA(**tank_params) # 값으로 대체
config['C_tank'] = c_w * rho_w * (math.pi * tank_params['r0']**2 * tank_params['H'])
config['T0_K'] =enex.C2K(config['T0'])



# 4. Simulation 실행 및 csv 저장

In [169]:
results_dataframe = run_simulation(gshpb_model, config, serv_sched)
results_dataframe.to_csv('gshpb_simulation_results_refactored.csv')

Simulating: 100%|██████████| 2880/2880 [00:12<00:00, 237.74it/s]


# 4. Visualization

## 4.1 Data import

In [170]:
df = pd.read_csv('gshpb_simulation_results_refactored.csv', index_col=0)

## 4.2 Plot code

In [171]:
# =============================================================================
# 5) (옵션) 플롯 시작부
#    - 한글 폰트 설정 등은 사용 OS 환경에 맞게 수동 설정
# =============================================================================
def plot_simple_graph(df_column, time, xlabel, ylabel, xmin = 0, xmax=24, Kelvin=False, color = 'dm.blue5', savepath = None):
    # 온도 데이터를 섭씨(°C)로 변환하여 플로팅
    if Kelvin:
        y = enex.K2C(df_column)
    else:
        y = df_column
    x = time * enex.s2h  # 초를 시간으로 변환

    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    
    # --- 마이너 틱 설정 ---

    # X축: 주 틱 사이를 5개의 간격으로 나눔 (마이너 틱 4개 생성)
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    # Y축: 주 틱 사이를 2개의 간격으로 나눔 (마이너 틱 1개 생성)
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    ax.plot(x, y, color=color, linewidth=0.5,)
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        else:
            ax.set_xticks(np.arange(0, xmax+1, 6))
    
    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    ax.set_ylim(np.nanmin(y)*0.95, np.nanmax(y)*1.05)
    # ax.set_ylim(50, 55)
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox = [0, 1, 0.02, 1])
    plt.savefig(f'{savepath}.png')
    dm.save_and_show(fig,)
    plt.close()

def plot_multi_graph(df_columns, legends, time, xlabel, ylabel, linestyles=None, colors=None, xmin=0, xmax=24, Kelvin=False, savepath=None):
    """
    여러 개의 데이터 열을 받아 하나의 그래프에 플로팅하는 함수.

    Args:
        *df_columns: 플로팅할 데이터 열들 (예: df['col1'], df['col2'], ...)
        labels (list or tuple): 각 데이터 열에 해당하는 레이블 리스트
        time (array-like): x축 시간 데이터
        ... (기존 인자들과 동일)
        colors (list or tuple, optional): 각 플롯에 사용할 색상 리스트. 기본값 None.
    """
    if len(df_columns) != len(legends):
        raise ValueError("데이터 컬럼의 개수와 레이블의 개수가 일치해야 합니다.")

    # --- 1. 그래프 기본 설정 ---
    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    x = time * enex.s2h  # 초를 시간으로 변환
    
    # 기본 색상 리스트 (colors 인자가 주어지지 않을 경우 사용)
    if colors is None:
        colors = ['dm.blue5', 'dm.orange5', 'dm.green5', 'dm.red5', 'dm.violet5',
                  'dm.brown5', 'dm.pink5', 'dm.gray5', 'dm.yellow5', 'dm.cyan5']
    
    # --- 2. 모든 데이터를 바탕으로 Y축 min/max 계산 ---
    global_min = np.inf
    global_max = -np.inf
    
    processed_ys = []
    for col in df_columns:
        y = enex.K2C(col) if Kelvin else col
        processed_ys.append(y)
        if not y.empty:
            global_min = min(global_min, np.nanmin(y))
            global_max = max(global_max, np.nanmax(y))
            
    # --- 3. 반복문을 통해 여러 데이터 플로팅 ---
    for i, y_data in enumerate(processed_ys):
        ax.plot(x, y_data, 
                color=colors[i % len(colors)],  # 색상 순환 사용
                linewidth=0.8, 
                label=legends[i],
                linestyle=linestyles[i % len(linestyles)] if linestyles is not None else ['-', '--', '-.', ':'][i % 4],
                )               # 범례를 위한 레이블 추가

    # --- 4. 축 및 레이아웃 설정 ---
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0:
        ax.set_xticks(np.arange(0, xmax + 1, 2 if xmax == 24 else 6))
    
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    
    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    ax.set_ylim(global_min * 0.95, global_max * 1.05) # 전체 데이터 기준 ylim 설정
    
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])
    
    ax.legend(ncol = 4) # 범례 표시
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=[0, 1, 0.02, 1])
    
    if savepath:
        plt.savefig(f'{savepath}.png')
        
    dm.save_and_show(fig)
    plt.close()

In [172]:
folder_path = 'figure/HeatPump_model'

plot_simple_graph(df['is_on'], config['time'],'Time (h)', 'Heater on/off [ - ]', xmax =24*2, color='dm.green5', savepath = f'{folder_path}/is_on')

plot_simple_graph(df['g_conv'], config['time'],'Time (h)', '$R_g(t)$ transient [(m·K)/W]', xmax =24*2, color='dm.red5', savepath = f'{folder_path}/g_conv')

plot_simple_graph(df['dV_w_serv']*enex.m32L/enex.s2m, config['time'],'Time (h)', 'Water use schedule (L/min)', xmax =24*2, color='dm.violet5', savepath = f'{folder_path}/dV_w_serv')
plot_simple_graph(df['T_w_tank'], config['time'],'Time (h)', 'Tank water temp. [°C]', xmin = 0, xmax =24*2, color='dm.blue5', savepath = f'{folder_path}/T_w_tank')
plot_simple_graph(df['T_bh'], config['time'],'Time (h)', 'Bore hole wall temp. [°C]', xmax =24*2, color='dm.violet5', savepath = f'{folder_path}/T_bh', Kelvin=True)
plot_multi_graph((df['T_f_bh_in'], df['T_f_bh_out']), legends = ['Bore hole inlet water', 'Bore hole outlet water'], time=config['time'], xlabel='Time (h)', ylabel='Temperature [°C]', xmax=24*2, Kelvin=True, savepath=f'{folder_path}/T_f_bh_in_out')

plot_simple_graph(df['E_cmp'], config['time'],'Time (h)', 'Compressor power Input (W)', xmax =24*2, color='dm.yellow7', savepath = f'{folder_path}/E_cmp')
plot_simple_graph(df['COP'], config['time'],'Time (h)', 'COP [°C]', xmax =24*2, color='dm.violet5', savepath = f'{folder_path}/COP')

plot_simple_graph(df['Xc_g'], config['time'],'Time (h)', 'Ground exergy consum rate [W]', xmax =24*2, color='dm.orange5', savepath = f'{folder_path}/Xc_g')


# 3. 시뮬레이션

In [ ]:

nan_arr = [np.nan] * tN
results = {
    # 온도 및 열량
    'T_bh': nan_arr.copy(), 'T_f_bh': nan_arr.copy(),
    'T_f_bh_in': nan_arr.copy(), 'T_f_bh_out': nan_arr.copy(),
    'Q_bh_unit': nan_arr.copy(),

    # 각 컴포넌트별 엑서지 (입력, 출력, 파괴량)
    'Xin_g': nan_arr.copy(), 'Xout_g': nan_arr.copy(), 'Xc_g': nan_arr.copy(),
    'Xin_GHE': nan_arr.copy(), 'Xout_GHE': nan_arr.copy(), 'Xc_GHE': nan_arr.copy(),
    'Xin_HX': nan_arr.copy(), 'Xout_HX': nan_arr.copy(), 'Xc_HX': nan_arr.copy(),
    'Xin_r': nan_arr.copy(),  'Xout_r': nan_arr.copy(),  'Xc_r': nan_arr.copy(),
    'Xin_tank': nan_arr.copy(), 'Xout_tank': nan_arr.copy(), 'Xc_tank': nan_arr.copy(),
    'Xin_mix': nan_arr.copy(), 'Xout_mix': nan_arr.copy(), 'Xc_mix': nan_arr.copy(),
    'E_cmp': nan_arr.copy(), 'E_pmp': nan_arr.copy(), 
    # 냉매 엑서지
    'x1': nan_arr.copy(), 'x2': nan_arr.copy(), 'x3': nan_arr.copy(), 'x4': nan_arr.copy(),
    'T_w_tank': nan_arr.copy(),

    # 시스템 효율
    'X_eff': nan_arr.copy(),
    'is_on': nan_arr.copy(),
    'g': nan_arr.copy(),
}
# ... (기존 results 딕셔너리, Q_bh_unit_purse 등 초기화) ...

# =============================================================================
# 3) 시간 스텝 루프
# =============================================================================
for n in tqdm(range(tN), desc="Sim", leave=False):
    
    # --- (0-1) 현재 스텝의 손실량 계산 ---
    Q_env_loss = UA_tank * (T_w_tank[n] - T0)
    
    # alp 계산 및 급탕 사용 손실 계산
    den = max(1e-6, enex.C2K(T_w_tank[n]) - enex.C2K(T_w_sup))
    alp_n = min(1.0, max(0.0, (enex.C2K(T_w_serv) - enex.C2K(T_w_sup)) / den))
    dV_w_sup_tank = alp_n * dV_w_serv
    Q_use_loss = serv_sched_tot[n] * (enex.c_w * enex.rho_w * dV_w_sup_tank * (T_w_tank[n] - T_w_sup))
    
    total_loss = Q_env_loss + Q_use_loss

    # --- (0-2) 탱크 온도 기반 제어 로직 ★★★ ---
    # 현재 히트펌프가 켜져 있는지 여부를 이전 스텝 상태로부터 추정 (n=0일 때는 꺼져있다고 가정)
    is_on_prev_step = results['is_on'][n-1] if n > 0 else False
    
    # 켤지 말지 결정
    if T_w_tank[n] < T_w_tank_lower_bound:
        is_on_current_step = True # 하한 온도 아래로 떨어지면 무조건 켠다.
    elif T_w_tank[n] > T_w_tank_setpoint:
        is_on_current_step = False # 상한 온도를 넘으면 무조건 끈다.
    else: # 데드밴드 안에서는 이전 상태를 유지한다.
        is_on_current_step = is_on_prev_step

    # 현재 스텝에서 히트펌프가 감당해야 할 부하(q_load_n) 결정
    if is_on_current_step:
        q_load_n = -10000
    else:
        q_load_n = 0.0 # 히트펌프가 꺼져 있다면, 부하는 0이다.
    
    # --- (1) 최적 운전점 계산 ---
    # q_load_n을 바탕으로 최적 운전점을 찾는다. (q_load_n이 0이면 _off_result가 반환됨)
    ref_loop_result = GSHPB._find_ref_loop_optimal_operation(
        T_w_tank=T_w_tank[n],
        Q_load=q_load_n,
    )

    # ... (ref_loop_result is None 처리) ...
    results['is_on'][n] = ref_loop_result['is_on'] # is_on 상태 저장
    
    # --- (2) 탱크 온도 업데이트 (lumped capacitance) ---
    Q_tank_in = - ref_loop_result['Q_ref_tank'] # 히트펌프가 실제 공급한 열량
    Q_tank_net = Q_tank_in - total_loss
    
    # if n % 1000 == 0:
    #     print(f"Step {n}: T_w_tank = {T_w_tank[n]:.1f} °C, is_on = {is_on_current_step}, Q_tank_net = {Q_tank_net:.1f} W, total_loss = {total_loss:.1f} W")
    
    # 냉매 엑서지 (각 상태점) [J/kg]


    # --- (2) 보어홀 선형열속 Q_bh_unit (W/m) ---
    Q_bh_unit = (ref_loop_result['Q_ref_HX'] - GSHPB.E_pmp) / GSHPB.H_b

    # 펄스(증분) 기록
    if Q_bh_unit != Q_bh_unit_old:
        Q_bh_unit_purse[n] = Q_bh_unit - Q_bh_unit_old
        Q_bh_unit_old = Q_bh_unit


    # 경과시간 컨볼루션 (펄스별 tau)
    pulses_idx = np.flatnonzero(Q_bh_unit_purse) # dQ != 0 인 지점의 인덱스 array
    dQ = Q_bh_unit_purse[pulses_idx] # 각 펄스 크기 (W/m)
    tau = np.maximum(time[n] - time[pulses_idx], 0.0) # 각 펄스 이후 경과시간
    g_n = np.array([enex.G_FLS(t=t, ks=GSHPB.k_g, as_=GSHPB.alp_g, rb=GSHPB.r_b, H=GSHPB.H_b) for t in tau])
    results['g'][n] = np.sum(g_n) # convolution 된 g 값 기록
    
    dT_bh = np.dot(dQ, g_n)
    T_bh = GSHPB.Tg - dT_bh

    # 유체 온도 (보어홀-유체 열저항 반영)
    T_f_bh = T_bh - Q_bh_unit * GSHPB.R_b
    T_f_bh_in  = T_f_bh - Q_bh_unit * GSHPB.H_b / (2 * enex.c_w * enex.rho_w * GSHPB.dV_f)
    T_f_bh_out = T_f_bh + Q_bh_unit * GSHPB.H_b / (2 * enex.c_w * enex.rho_w * GSHPB.dV_f)

    # --- (3) 엑서지 계산 ---
    E_cmp = ref_loop_result['E_cmp']
    E_pmp = GSHPB.E_pmp
    Q_ref_tank = ref_loop_result['Q_ref_tank']  # 음수
    T3_K = ref_loop_result['T3']
    Q_ref_HX = ref_loop_result['Q_ref_HX']      # 양수
    T1_K = ref_loop_result['T1']

    # 일(Work)
    X_pmp = E_pmp
    X_cmp = E_cmp
    
    # 열(Heat)
    X_ref_int = Q_ref_tank * (1 - T0_K / T3_K)   # 냉매 -> 저탕조
    X_ref_HX  = Q_ref_HX  * (1 - T0_K / T1_K)   # GHE -> 냉매

    # 유체(Fluid)
    X_f_in  = enex.c_w * enex.rho_w * GSHPB.dV_f * ((T_f_bh_in  - T0_K) - T0_K * math.log(T_f_bh_in  / T0_K))
    X_f_out = enex.c_w * enex.rho_w * GSHPB.dV_f * ((T_f_bh_out - T0_K) - T0_K * math.log(T_f_bh_out / T0_K))

    # 지중 경계
    Q_bh = Q_bh_unit * GSHPB.H_b # [W]
    Xin_g = (1 - T0_K / GSHPB.Tg) * Q_bh  # 초기 지중온도 경계 [W]
    Xout_g = (1 - T0_K / T_bh) * Q_bh      # 보어홀 벽면온도 경계 [W]

    # 컴포넌트별 엑서지 입/출/파괴
    Xc_g            = Xin_g  - Xout_g

    Xin_GHE  = X_pmp + Xout_g + X_f_in
    Xout_GHE = X_f_out
    Xc_GHE   = Xin_GHE - Xout_GHE

    Xin_HX  = Xout_GHE
    Xout_HX = X_ref_HX + X_f_in
    Xc_HX   = Xin_HX - Xout_HX

    Xin_r, Xout_r   = X_cmp + X_ref_HX, X_ref_int
    Xc_r            = Xin_r - Xout_r

    Xin_tank         = abs(X_ref_int)
    Xout_useful_tank = abs(q_load_n) * (1 - T0_K / enex.C2K(T_w_tank[n]))
    Xc_tank          = Xin_tank - Xout_useful_tank

    # 시스템 엑서지 효율
    X_in_total = X_pmp + X_cmp
    X_eff = Xout_useful_tank / X_in_total if X_in_total > 0 else 0

    # 결과 기록
    results['x1'][n] = (ref_loop_result['h1'] - h0) - (ref_loop_result['s1'] - s0) * T0_K
    results['x2'][n] = (ref_loop_result['h2'] - h0) - (ref_loop_result['s2'] - s0) * T0_K
    results['x3'][n] = (ref_loop_result['h3'] - h0) - (ref_loop_result['s3'] - s0) * T0_K
    results['x4'][n] = (ref_loop_result['h4'] - h0) - (ref_loop_result['s4'] - s0) * T0_K
    
    results['T_bh'][n]        = T_bh
    results['T_f_bh'][n]      = T_f_bh
    results['T_f_bh_in'][n]   = T_f_bh_in
    results['T_f_bh_out'][n]  = T_f_bh_out
    results['Q_bh_unit'][n]        = Q_bh_unit
    
    results['E_cmp'][n]      = E_cmp
    results['E_pmp'][n]      = E_pmp if is_on_current_step else 0
    results['T_w_tank'][n]   = T_w_tank[n]

    results['Xin_g'][n]       = Xin_g
    results['Xout_g'][n]      = Xout_g
    results['Xc_g'][n]        = Xc_g

    results['Xin_GHE'][n]     = Xin_GHE
    results['Xout_GHE'][n]    = Xout_GHE
    results['Xc_GHE'][n]      = Xc_GHE

    results['Xin_HX'][n]      = Xin_HX
    results['Xout_HX'][n]     = Xout_HX
    results['Xc_HX'][n]       = Xc_HX

    results['Xin_r'][n]       = Xin_r
    results['Xout_r'][n]      = Xout_r
    results['Xc_r'][n]        = Xc_r

    results['Xin_tank'][n]    = Xin_tank
    results['Xout_tank'][n]   = Xout_useful_tank
    results['Xc_tank'][n]     = Xc_tank

    results['Xc_mix'][n]      = np.nan

    results['X_eff'][n]       = X_eff

    # 순환 유체 입력온도 업데이트 (°C 입력)
    GSHPB.T_f_bh_in = T_f_bh_in
    if n < tN - 1:
        T_w_tank[n+1] = T_w_tank[n] + (Q_tank_net / C_tank) * dt

Sim:   0%|          | 0/14400 [00:00<?, ?it/s]

In [ ]:
# =============================================================================
# 5. 메인 실행 블록
# =============================================================================
if __name__ == '__main__':
    
    # --- 1. 시뮬레이션 설정 (중앙 관리) ---
    config = {
        'simulation_period_sec': 10 * enex.d2h * enex.h2s, # 10일
        'dt': 1 * enex.m2s, # 1분
        'T_w_tank_setpoint': 60.0,
        'T_w_tank_lower_bound': 50.0,
        'T_w_tank_init': 55.0,
        'T0': 0.0,
        'T_w_serv': 45.0,
        'T_w_sup': 15.0,
        'dV_w_serv_m3s': 8 * enex.L2m3 / enex.m2s,
        'results_keys': [
            'T_bh', 'T_f_bh_in', 'T_w_tank', 'Q_bh_unit', 'is_on', 'E_cmp', 'E_pmp',
            'Xin_g', 'Xout_g', 'Xc_g',
            'Xin_GHE', 'Xout_GHE', 'Xc_GHE',
            'Xin_HX', 'Xout_HX', 'Xc_HX',
            'Xin_r', 'Xout_r', 'Xc_r',
            'Xin_tank', 'Xout_tank', 'Xc_tank',
            'Xc_mix',
            'X_eff'
            # 필요한 다른 결과 키들 추가...
        ]
    }
    
    # --- 2. 시간 및 부하 프로파일 생성 ---
    config['time'] = np.arange(0, config['simulation_period_sec'], config['dt'])
    tN = len(config['time'])
    
    serv_sched_hourly = [0]*6 + [1]*1 + [0]*11 + [1]*1 + [0]*5 # 6-7h, 18-19h
    serv_sched_minutely = np.repeat(serv_sched_hourly, int(enex.h2s / config['dt']))
    num_days = int(np.ceil(tN / len(serv_sched_minutely)))
    serv_sched = np.tile(serv_sched_minutely, num_days)[:tN]

    # --- 3. 모델 및 기타 파라미터 초기화 ---
    gshpb_model = enex.GroundSourceHeatPumpBoiler2(
        disp_cmp=0.00005, eta_cmp_isen=0.7, T_f_bh_in=15.0, Tg=15.0
    )
    
    tank_params = {'r0': 0.2, 'H': 0.8}
    config['UA_tank'] = 3.5 # enex.calc_simple_tank_UA(**tank_params) # 값으로 대체
    config['C_tank'] = c_w * rho_w * (math.pi * tank_params['r0']**2 * tank_params['H'])
    config['T0_K'] =enex.C2K(config['T0'])
    
    # --- 4. 시뮬레이션 실행 ---
    results_dataframe = run_simulation(gshpb_model, config, serv_sched)
    
    # --- 5. 결과 저장 및 시각화 ---
    results_dataframe.to_csv('gshpb_simulation_results_refactored.csv')

# 4. CSV 저장

In [30]:
# =============================================================================
# 4) CSV 저장
# =============================================================================
results_df = pd.DataFrame(results)
results_df.to_csv('GSHPB_exergy_simulation_results.csv', index=False)

NameError: name 'results' is not defined

# 5. Plot

## 5.1 Data import

In [77]:
df = pd.read_csv('GSHPB_exergy_simulation_results.csv')